# Finance - Capital Asset Pricing Model & Portafolio

---
---

## Capital Asset Pricing Model (CAPM)

CAPM is one of the most important models in Finance. The CAPM model describes the **relationship between the expected return and risks of securities**.


### Risk Free Asset Return ($r_{f}$)

- **Model Assumptions**: The model assumes the existance of a risk free asset with a 0 standard deviation.
- If buyer is extremely risk averse, but the asset to protect your money, but the down side is that you earn a low return $r_{f}$.

### Market Overall Portafolio Return ($r_{m}$)

- The Market Portafolio includes all the securities in the market, a good example is the S&P500 (Standard & Poor's 500 Index).


### CAPM Formula

$$r_{i} = r_{f} + \beta_{i} (r_{m} - r_{f})$$

$$r_{i} = \text{Expected Return of the Stock } i$$
$$r_{f} = \text{Risk Free Rate of Return }$$
$$r_{m} = \text{Market Overall Return }$$
$$\beta_{i} = \text{Beta between the Market & the Stock } i$$
$$(r_{m} - r_{f}) = \text{Risk Premium (incentive to invest with more risk)}$$


### The Beta and Alpha Values

- The Beta represents the slope of the regression line (market return VS stock return).
- The Beta is a **measure of Volatility or Systematic Risk of a Stock or a Portafolio compared to the entire market** (S&P500), and describes the relationship between the systematic risk and the expected return for the assets. 
    - $\beta_{i} = 1.0$: Indicates that the stock or portafolio price is strongly correlated with the market.
    - $\beta_{i} > 1.0 \textit{ (Aggressive)}$: Indicates that the **stock price is more volatile than the market**. For example, Tesla stock beta is 1.26, meaning that is 26% more volatile than the market.
    - $\beta_{i} < 1.0 \textit{ (Defensive)}$: Indicates that the **stock price is less volatile than the market**.


- Alpha describes the strategy's ability to beat the market (S&P500), **indicates the excess return** or “abnormal rate of return”. 
- For example, a positive alpha value of 0.175 means that the portfolio’s return exceeded the benchmark S&P500  by 17%.

### Apply CAPM in Practice

Let's assume that we are going to apply the CAPM model for the Apple Stock and using the following based assumptions required for this process.

- $r_{f} = 0%$ *(yield from a 10 years US government bond could also be used)*
- $r_{m} = 12.4%$ *(S&P500 rate of return)*
- $\beta_{AAPL} = 1.11$

Therefore, the expected return for Apple Stock $r_{AAPL}$ is going to be estimated as follows:

- $r_{AAPL} = \text{0%} + 1.11·(0.124 - \text{0%}) = 0.137 = \text{13.7%}$

- **Recipy**:
    - **Step 1**: Normalize the Prices based on the initial price.
    - **Step 2**: Calculate the Daily Returns.
    - **Step 3**: Estimate the Beta for the Stock. This value can be estimated using a lineal regression between the market and the stock, or retrieve the beta value from yahoo finance.
    - **Step 4.1 (Single Stock Return)**: Apply the CAPM formula to the individual Stock.
    - **Step 4.2 (Portafolio Return with Multiple Stocks)**: Estimate the Individual Beta for each Stock and them apply the CAPM formula to each of the individual Stocks. Then finally, we define the weight of each stock inside the portafolio and multiple the expected return of each individual stock with their respective weight and sum all the values to calculate the Expected Return Based on CAPM for the portfolio.

```python
# RECIPY: Applied with Python


# STEP 1: Normalize the Prices
# Normalize stock price based on 1st price (Single Stock)
def normalize_single_stock(df):
  df = df.copy()
  df['Stock'] = df['Stock'] / df['Stock'][0]
  return df

# Normalize stock prices based on 1st price (Multiple Stocks)
def normalize_stocks(df):
  df = df.copy()
  for i in df.columns[1:]: # The first col is the date
    df[i] = df[i] / df[i][0]
  return df


# STEP 2: Estimate Daily Returns (Multiple Stocks)
def daily_returns_multiple_stocks(df):
  df = df.copy()
  for i in df.columns[1:]: # The first col is the date
    daily_return_stock_i = f"daily_return_{i}"    
    df[daily_return_stock_i] = df[i].pct_change()
  return df.dropna()


# STEP 3: Estimate the Beta for the Multiple Stocks
import numpy as np

# Store betas and alphas
beta = {}
alpha = {}

# Loop on every stock daily return
for i in df_daily_return.columns:
    
  # Ignoring the date and S&P500 Columns 
  if (i != 'Date') and (i != 'sp500'):
    # Fit a polynomial between each stock and the S&P500 (degree=1)
    b, a = np.polyfit(df_daily_return['sp500'], df_daily_return[i], 1)
    
    # Append results to dicts
    beta[i] = b
    alpha[i] = a


# STEP 4: Estimate Portafolio Expected Return with CAPM
# Obtain a list of all stock names
keys = list(beta.keys())

# Define the expected return dictionary
ER = {}

# Risk free rate
rf = 0
# Market risk expected return annualized (252 trading days in 1 year)
rm = df_daily_return['sp500'].mean() * 252

# Calculate return for every stock using CAPM
for i in keys:
  ER[i] = rf + (beta[i] * (rm-rf))

# Assume equal weights in the portfolio
portfolio_weights = 1/len(keys) * np.ones(len(keys)) 

# Calculate the portfolio return 
ER_portfolio = sum(list(ER.values()) * portfolio_weights)

# Print the results
print(f'Expected Return Based on CAPM for the portfolio is {ER_portfolio}%\n')


############################################################################################
#                                   Several Useful TIPS
############################################################################################

# Create Random Portfolio Weights
# Set seed
np.random.seed()

# Create random weights for the stocks and then normalize them
# Include all stocks and market and drop the date column
weights = np.array(np.random.random(len(df.columns[1:])))

# Ensure that the sum of all weights equals 1
weights = weights / np.sum(weights)


# Estimate Portafolio Statistical Metrics
# Let's put all of these statistical measures
def portfolio_statistical_analysis(weights):   
  # Get the portfolio allocation for the specified weights (sent as arguments)
  df_portfolio = portfolio_allocation(stocks_df, weights)
  
  # Cummulative return of the portfolio (Note: look for the last net worth of the portfolio compared to 1st value)
  cummulative_return = ((df_portfolio['portfolio daily worth in $'][-1:] - df_portfolio['portfolio daily worth in $'][0])/ \ 
                                                      df_portfolio['portfolio daily worth in $'][0]) * 100
  
  # Daily change of every stock in the portfolio (Note: dropped the date, portfolio daily worth and daily % returns)
  df_portfolio_daily_return = df_portfolio_daily_return.pct_change(1) 
  
  # Portfolio Expected Annual Return 
  expected_portfolio_return = np.sum(df_portfolio_daily_return.mean() * weights * 252)
  
  # Portfolio Expected Volatility
  covariance = df_portfolio_daily_return.cov() * 252 
  expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))
  
  # Portfolio sharpe ratio
  sharpe_ratio = expected_portfolio_return/expected_volatility 

  return cummulative_return.values[0], expected_portfolio_return, expected_volatility, sharpe_ratio

############################################################################################

# PORTFOLIO OPTIMIZATION WITH RANDOM ALLOCATION USING MONTE CARLO SIMULATIONS

# Create random weights and analayze the portfolio to find the optimal weight allocation
number_of_trials = 100

# Store the weights
possible_weights_runs = np.zeros((number_of_trials, 9))   # number of stocks + market = 9 cols

# Placeholder to store the sharpe ratios
sharpe_ratio_runs = np.zeros(number_of_trials)

# Placeholder to store the returns
expected_portfolio_returns_runs = np.zeros(number_of_trials)

# Placeholder to store the volatility
volatility_runs = np.zeros(number_of_trials)

# Placeholder to store the cummulative returns
cummulative_returns_runs = np.zeros(number_of_trials)

for i in range(number_of_trials):
    print(i)
    # Random Weights
    weights = np.array(np.random.random(9))
    weights = weights / np.sum(weights)
    
    # Store the weights
    possible_weights_runs[i,:] = weights
    
    # Store the sharpe ratio, return and volatility
    cummulative_returns_runs[i], expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i]  = portfolio_statistical_analysis(weights)

############################################################################################

# PORFOLIO OPTIMIZATION USING OPTIMIZERS

# SciPy is an open-source ecosystems for mathematical calculations 
# We will minimize the cost function using Sequential Least Squares Programming (SLSQP)

from scipy.optimize import minimize

# Since optimization works as a minimizing function, we take negative of sharpe ratio and then try minimize it.

# Define a function that calculates the negative Sharpe ratio
def calculate_negative_sharpe_func(weights):
  cum_return, exp_portfolio_return, volatility, sharpe_ratio = portfolio_statistical_analysis(weights)
  return sharpe_ratio * -1

# Function to define optimization constraints (make sure sum of all weights add to 1)
def optimization_constraints_func(weights):
  return np.sum(weights) - 1

# Function to obtain the "volatility" for a given set of portfolio weights
def calculate_volatility_func(weights):
  cum_return, exp_portfolio_return, volatility, sharpe_ratio = portfolio_statistical_analysis(weights)
  return volatility

# Function to get the return for a particular weight
def calculate_return_func(weights):
  cum_return, exp_portfolio_return, volatility, sharpe_ratio = portfolio_statistical_analysis(weights)
  return exp_portfolio_return

# Constraints to pass to the optimizer
optimization_constraint = ({'type':'eq','fun': optimization_constraints_func})

# Lower and upper bounds for weights
bounds = ((0, 1), (0, 1), (0, 1), (0, 1),(0, 1), (0, 1), (0, 1), (0, 1),(0,1))

# Initial weight assumption
initialization = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]

# Optimization Function
# Optimization Algorithms is SLSQP stands for Sequential Least Squares Programming (SLSQP)   
optimization_results = minimize(calculate_negative_sharpe_func, initialization, method='SLSQP', \
                                 bounds=bounds, constraints=optimization_constraint)

optimized_weights = optimization_results.x

# Analyze the portfolio based on the obtained weights
cummulative_return_SLSQP, portfolio_return_value_SLSQP, vol_value_SLSQP, sharpe_ratio_SLSQP = portfolio_statistical_analysis(optimized_weights)

print('Best Portfolio metrics based on SLSQP optimizer:')
print('  - Optimal Cummulative return of the portfolio = {}%'.format(cummulative_return_SLSQP))
print('  - Portfolio Expected return per year = {:.02f}%'.format(portfolio_return_value_SLSQP * 100))
print('  - Volatility of the portfolio = {:.02f}%'.format(vol_value_SLSQP * 100))
print('  - Sharpe ratio of the portfolio = {}'.format(sharpe_ratio_SLSQP))

# PLOTTING MARKOWITZ EFFICIENT FRONTIER
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

# The efficient frontier represents a set of portfolios that produces the highest return for any risk level 
# the efficient fronitier represents the portfolio with the least amount of risk (volatility)(x-axis) for any given expected return (y-axis)
# The returns range from 0 to 0.35 so we will obtain x number of y-positions (x = 50) to calculate the optimal volatility for that return
frontier_y = np.linspace(0, 0.35, 50)

frontier_volatility = []

# Calculate the optimal volatility for each return
for possible_return in frontier_y:

    # Conditions for optimization
    condition = ({'type':'eq', 'fun': optimization_constraints_func},
                 {'type':'eq', 'fun': lambda w: calculate_return_func(w) - possible_return})
    
    result = minimize(calculate_volatility_func, initialization, method = 'SLSQP', bounds = bounds, constraints = condition)
    
    frontier_volatility.append(result['fun'])
    
# Plot the efficient frontier
fig = px.scatter(df_optimize, x = 'Volatility', y = 'Portfolio_Return', color = 'Sharpe_Ratio' )
fig.add_trace(go.Scatter(x = frontier_volatility, y = frontier_y.tolist(), name = 'Frontier'))
fig.add_trace(go.Scatter(x = [vol_value], y = [portfolio_return_value], mode = 'markers', name = 'Optimal Point', marker_color = 'black'))
fig.show()
```

$$$$

------
------

In [1]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [5]:
# Import data
stocks_df = pd.read_csv("stock.csv")

# Sort the data by date
stocks_df = stocks_df.sort_values(by = ['Date'])

In [8]:
# Define Portafolio allocation function

# Let's create random portfolio weights
# Set seed
np.random.seed(101)

# Create random weights for the stocks and normalize them
weights = np.array(np.random.random(9))

# Ensure that the sum of all weights are = 1
weights = weights / np.sum(weights) 

# Lets assume we have $1,000,000 to be invested and we will allocate this fund based on the weights of the stocks
# We will create a function that takes in the stock prices along with the weights and retun:
# (1) Daily value of each individual securuty in $ over the specified time period
# (2) Overall daily worth of the entire portfolio 
# (3) Daily return 


def normalize(df):
    df_normalize = df.copy()
    for stock in df_normalize.columns[1:]:
        df_normalize[stock] = df_normalize[stock]/df_normalize[stock][0]
    
    return df_normalize


def portfolio_allocation(df, weights):
  df_portfolio = df.copy()
  
  # Normalize the stock avalues 
  df_portfolio = normalize(df_portfolio)
  
  for counter, stock in enumerate(df_portfolio.columns[1:]):
    df_portfolio[stock] = df_portfolio[stock] * weights[counter]
    df_portfolio[stock] = df_portfolio[stock] * 1000000

  df_portfolio['portfolio daily worth in $'] = df_portfolio[df_portfolio != 'Date'].sum(axis = 1)
  
  df_portfolio['portfolio daily % return'] = 0.0000

  for i in range(1, len(stocks_df)):
    
    # Calculate the percentage of change from the previous day
    df_portfolio['portfolio daily % return'][i] = ( (df_portfolio['portfolio daily worth in $'][i] - df_portfolio['portfolio daily worth in $'][i-1]) / df_portfolio['portfolio daily worth in $'][i-1]) * 100 
  
  # set the value of first row to zero, as previous value is not available
  df_portfolio['portfolio daily % return'][0] = 0
  return df_portfolio

# Get the portfolio allocation for the randomly selected weights
df_portfolio = portfolio_allocation(stocks_df, weights)
df_portfolio

/Users/NicolasEnriquez/opt/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500,portfolio daily worth in $,portfolio daily % return
0,2012-01-12,109213.072967,120690.407490,6022.010143,36275.090893,1.449291e+05,176360.729910,6.492024e+04,188990.104235,152599.209597,1.000000e+06,0.000000
1,2012-01-13,108803.583155,119235.914699,6012.013247,36933.006803,1.469804e+05,175002.982837,5.237283e+04,187594.381427,151844.160487,9.847792e+05,-1.522076
2,2012-01-17,110070.940263,120258.850187,6048.001354,36633.954117,1.496495e+05,175823.488543,6.112844e+04,188671.942595,152383.655881,1.000669e+06,1.613507
3,2012-01-18,111213.896735,119971.149581,6063.996069,38069.407013,1.560585e+05,176868.668340,6.161103e+04,189971.620676,154076.322712,1.013905e+06,1.322705
4,2012-01-19,110861.418590,120770.317932,6081.990122,38278.743893,1.601857e+05,176331.426972,6.149613e+04,191970.659033,154837.252810,1.020814e+06,0.681427
...,...,...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,798707.600095,278558.118654,5968.027783,50001.606220,2.640267e+06,122539.200612,3.412668e+06,887941.003611,391983.847760,8.588634e+06,0.879605
2155,2020-08-06,826573.895966,275233.575119,5966.028443,55205.122968,2.656718e+06,123193.660569,3.423147e+06,903902.858806,394503.400529,8.664444e+06,0.882669
2156,2020-08-07,806327.319584,271749.212299,6002.016551,56909.729263,2.609318e+06,122060.571958,3.338417e+06,900522.495854,394753.132817,8.506059e+06,-1.827981
2157,2020-08-10,818047.125846,286757.593922,6038.004858,64744.906663,2.593418e+06,124160.687803,3.259961e+06,901492.611827,395835.632149,8.450456e+06,-0.653686


In [9]:
# CALCULATE PORTFOLIO STATISTICAL ANALYSIS KEY METRICS
def portfolio_statistical_analysis(weights):

  # Get the portfolio allocation for the specified weights (sent as arguments)
  df_portfolio = portfolio_allocation(stocks_df, weights)
  
  # Cummulative return of the portfolio (Note that we now look for the last net worth of the portfolio compared to it's start value)
  cummulative_return = ((df_portfolio['portfolio daily worth in $'][-1:] - df_portfolio['portfolio daily worth in $'][0])/ df_portfolio['portfolio daily worth in $'][0]) * 100
  
  # Daily change of every stock in the portfolio (Note that we dropped the date, portfolio daily worth and daily % returns) 
  df_portfolio_daily_return = df_portfolio.drop(columns = ['Date', 'portfolio daily worth in $', 'portfolio daily % return'])
  df_portfolio_daily_return = df_portfolio_daily_return.pct_change(1) 
  
  # Portfolio Expected Annual Return 
  expected_portfolio_return = np.sum(df_portfolio_daily_return.mean() * weights * 252)
  
  # Portfolio Expected Volatility
  covariance = df_portfolio_daily_return.cov() * 252 
  expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))
  
  # Portfolio sharpe ratio
  sharpe_ratio = expected_portfolio_return/expected_volatility 

  return cummulative_return.values[0], expected_portfolio_return, expected_volatility, sharpe_ratio



cm, er, ev, sr = portfolio_statistical_analysis(weights)
print(f"""
Portfolio Cummulative Return: {cm:.3f}%
Expected portfolio returns: {er:.3f}%
Expected volatility: {ev:.3f}%
Sharpe Ratio = {sr:.3f}%
""")
  

/Users/NicolasEnriquez/opt/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)



Portfolio Cummulative Return: 725.669%
Expected portfolio returns: 0.206%
Expected volatility: 0.202%
Sharpe Ratio = 1.024%



In [10]:
# PORTFOLIO OPTIMIZATION RANDOM ALLOCATION USING MONTE CARLO
number_of_trials = 100

# Placeholder to store the weights
possible_weights_runs = np.zeros((number_of_trials, 9))

# Placeholder to store the sharpe ratios
sharpe_ratio_runs = np.zeros(number_of_trials)

# Placeholder to store the returns
expected_portfolio_returns_runs = np.zeros(number_of_trials)

# Placeholder to store the volatility
volatility_runs = np.zeros(number_of_trials)

# Placeholder to store the cummulative returns
cummulative_returns_runs = np.zeros(number_of_trials)

for i in range(number_of_trials):
    print(i)
    # Random Weights
    weights = np.array(np.random.random(9))
    weights = weights / np.sum(weights)
    
    # Store the weights
    possible_weights_runs[i,:] = weights
    
    # Store the sharpe ratio, return and volatility
    cummulative_returns_runs[i], expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i]  = portfolio_statistical_analysis(weights)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [11]:
# Create a DataFrame with all volatilties, portfolio returns, and sharpe ratios
df_optimize = pd.DataFrame({'Volatility': volatility_runs.tolist(), 'Portfolio_Return': expected_portfolio_returns_runs.tolist(), 'Sharpe_Ratio': sharpe_ratio_runs.tolist() })
df_optimize

,Volatility,Portfolio_Return,Sharpe_Ratio
0,0.197010,0.176645,0.896629
1,0.201965,0.182434,0.903296
2,0.217355,0.213206,0.980912
3,0.236262,0.235079,0.994990
4,0.237566,0.220150,0.926691
...,...,...,...
95,0.217906,0.239912,1.100989
96,0.204533,0.224994,1.100040
97,0.248088,0.256235,1.032835
98,0.249794,0.191775,0.767731


In [13]:
# PORFOLIO OPTIMIZATION USING OPTIMIZERS

# We will minimize the cost function using Sequential Least Squares Programming (SLSQP)

from scipy.optimize import minimize

# Since optimization works as a minimizing function, we take negative of sharpe ratio and then try minimize it.

# Define a function that calculates the negative Sharpe ratio
def calculate_negative_sharpe_func(weights):
  cum_return, exp_portfolio_return, volatility, sharpe_ratio = portfolio_statistical_analysis(weights)
  return sharpe_ratio * -1

# Function to define optimization constraints (make sure sum of all weights add to 1)
def optimization_constraints_func(weights):
  return np.sum(weights) - 1

# Function to obtain the "volatility" for a given set of portfolio weights
def calculate_volatility_func(weights):
  cum_return, exp_portfolio_return, volatility, sharpe_ratio = portfolio_statistical_analysis(weights)
  return volatility

# Function to get the return for a particular weight
def calculate_return_func(weights):
  cum_return, exp_portfolio_return, volatility, sharpe_ratio = portfolio_statistical_analysis(weights)
  return exp_portfolio_return

In [14]:
# Constraints to pass to the optimizer
optimization_constraint = ({'type':'eq','fun': optimization_constraints_func})

# Lower and upper bounds for weights
bounds = ((0, 1), (0, 1), (0, 1), (0, 1),(0, 1), (0, 1), (0, 1), (0, 1),(0,1))

# Initial weight assumption
initialization = [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]

In [15]:
# Optimization Function (Slow!!!)
# Optimization Algorithms is SLSQP stands for Sequential Least Squares Programming (SLSQP)   
optimization_results = minimize(calculate_negative_sharpe_func, initialization, method = 'SLSQP', bounds = bounds, constraints = optimization_constraint)

/Users/NicolasEnriquez/opt/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [16]:
optimized_weights = optimization_results.x
optimized_weights

array([2.57854617e-01, 7.37816636e-06, 7.37816636e-06, 7.37816636e-06,
       5.03402884e-01, 7.37816636e-06, 2.38735059e-01, 7.43940476e-06,
       7.37816636e-06])

In [17]:
# Analyze the portfolio based on the obtained weights
cummulative_return_SLSQP, portfolio_return_value_SLSQP, vol_value_SLSQP, sharpe_ratio_SLSQP = portfolio_statistical_analysis(optimized_weights)

In [21]:
print('Best Portfolio metrics based on SLSQP optimizer:')
print('- Optimal Cummulative return of the portfolio = {}%'.format(cummulative_return_SLSQP))
print('- Portfolio Expected return per year = {:.02f}%'.format(portfolio_return_value_SLSQP * 100))
print('- Volatility of the portfolio = {:.02f}%'.format(vol_value_SLSQP * 100))
print('- Sharpe ratio of the portfolio = {}'.format(sharpe_ratio_SLSQP))

Best Portfolio metrics based on SLSQP optimizer:
- Optimal Cummulative return of the portfolio = 2130.292859907244%
- Portfolio Expected return per year = 40.53%
- Volatility of the portfolio = 27.02%
- Sharpe ratio of the portfolio = 1.4999897176766592


In [22]:
# PLOTTING MARKOWITZ EFFICIENT FRONTIER

# The efficient frontier represents a set of portfolios that produces the highest return for any risk level 
# the efficient fronitier represents the portfolio with the least amount of risk (volatility)(x-axis) for any given expected return (y-axis)
# The returns range from 0 to 0.35 so we will obtain x number of y-positions (x = 50) to calculate the optimal volatility for that return
frontier_y = np.linspace(0, 0.35, 50)

frontier_volatility = []

# Calculate the optimal volatility for each return (VERY SLOW !!!)
for possible_return in frontier_y:

    # Conditions for optimization
    condition = ({'type':'eq', 'fun': optimization_constraints_func},
                 {'type':'eq', 'fun': lambda w: calculate_return_func(w) - possible_return})
    
    result = minimize(calculate_volatility_func, initialization, method = 'SLSQP', bounds = bounds, constraints = condition)
    
    frontier_volatility.append(result['fun'])

In [ ]:
# Plot the efficient frontier
fig = px.scatter(df_optimize, x = 'Volatility', y = 'Portfolio_Return', color = 'Sharpe_Ratio' )
fig.add_trace(go.Scatter(x = frontier_volatility, y = frontier_y.tolist(), name = 'Frontier'))
# fig.add_trace(go.Scatter(x = [vol_value], y = [portfolio_return_value], mode = 'markers', name = 'Optimal Point', marker_color = 'black'))
fig.show()